In [83]:
import numpy as np
from cmath import sqrt

In [2]:
ab = '10'

In [68]:
def invert_dict(d):
    return {v:k for k,v in d.items()}

In [90]:
zero = np.matrix('1;0')
one = np.matrix('0;1')

In [207]:
def kron(*matrices):
    assert len(matrices) > 0, "Cannot perform Kronecker product on no matrices"
    r = matrices[0]
    for mat in matrices[1:]:
        r = np.kron(r, mat)
    return r

In [208]:
class Q(object):
    
    def __init__(self, prob_map=None):
        # prob map keys are classical states
        # prob map values are (complex) amplitudes
        if prob_map is None:
            prob_map = {zero: 1+0j}
        err = abs(1 - sum(p**2 for p in prob_map.values()))
        assert err < 0.00001, "Squared amplitudes must sum to 1"
        self.prob_map = prob_map
        
    def __add__(self, other):
        """ Superimposes two quantum states. """
        pself = self.prob_map
        pother = other.prob_map
        # the new set of state outcomes is the concatenation of both quantum states
        states = pself.keys() + pother.keys()
        # the new probability map is the sum of the amplitudes of each state
        # (if the state is not existent in one map, its amplitude is 0)
        pnew = {state: pself.get(state, 0) + pother.get(state, 0) for state in states}
        # compute the normalization factor
        norm_factor = sqrt(sum(p**2 for p in pnew.values()))
        # compute the new probability map, with each amplitude divided by the norm factor
        norm_pnew = {state: prob / norm_factor for (state, prob) in pnew.iteritems()}
        return Q(norm_pnew)
    
    def apply_gate(self, gate):
        """ Apply a gate to the current state. The given `gate` should be an nxn matrix
        (where n is the number of qubits in the current state)"""
        gate_rows, gate_cols = gate.shape
        assert gate_rows == gate_cols, "Gate must be square"
        example_state = self.prob_map.iterkeys().next() # get one value (a state) out of the prob map
        state_rows, state_cols = example_state.shape
        assert state_cols == 1, "Somehow, this state has more than one column"
        assert state_rows == gate_rows, "Gate must have order equal to number of rows in the quantum state"
        new_superposition = {gate * state: prob for (state, prob) in self.prob_map.iteritems()}
        return Q(new_superposition)
    
    def __repr__(self):
        return "<Q: {:}>".format(self.prob_map)

In [219]:
X = np.matrix('0 1; 1 0')
Z = np.matrix('1 0; 0 -1')
H = 1/sqrt(2) * (X + Z)

def get_entangled_pair():
    return Q({
        kron(zero, zero): 1/sqrt(2),
        kron(one, one): 1/sqrt(2)
    })

In [193]:
def gate(unitary, size, qubit_index):
    """ Given a 2x2 gate (`unitary`), this generates a matrix which is the identity matrix of order `size`
    except for the 2x2 area corresponding to the desired qubit index. """
    eye = (1+0j) * np.eye(size)
    idx = qubit_index
    eye[np.ix_([idx, idx+1], [idx, idx+1])] = unitary
    return eye

In [236]:
def cnot_gate(size, control_qubit_index, target_qubit_index):
    return gate(X, size, target_qubit_index)

cnot_gate(4, 0, 2).astype(dtype=float)

/usr/local/lib/python2.7/site-packages/IPython/kernel/__main__.py:4: ComplexWarning: Casting complex values to real discards the imaginary part


array([[ 1.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  1.],
       [ 0.,  0.,  1.,  0.]])

In [234]:
# 1. start with an entangled pair. Alice owns qubit 1 and Bob owns qubit 2
state = get_entangled_pair()
state

# 2. Alice wants to send ab = 10
# Since a = 1, Alice applies Z to her qubit.
alice_z = gate(Z, 4, 0)   # rotation matrix is 4x4, Alice's qubit is index 0
state = state.apply_gate(alice_z)
# If b = 1, Alice would apply X to her qubit.
state

# 3. Alice sends her qubit to Bob.
state

# 4. Bob applies CNOT with Alice's qubit as control.
state = state.apply_gate(cnot_gate(4, 0, 2))  # 2 qubit gate, qubit 0 is control, qubit 1 is target

# 5. Bob applies H to Alice's qubit.
alice_h = gate(H, 4, 0)
state = state.apply_gate(alice_h)

# 6. Bob measures the qubits to get message ab.
state

<Q: {matrix([[ 0.70710678+0.j],
        [ 0.70710678+0.j],
        [ 0.00000000+0.j],
        [ 0.00000000+0.j]]): (0.7071067811865475+0j), matrix([[ 0.+0.j],
        [ 0.+0.j],
        [ 1.+0.j],
        [ 0.+0.j]]): (0.7071067811865475+0j)}>